In [2]:
pip install opencv-python

  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl (12.9 MB)

  Attempting uninstall: numpy

    Found existing installation: numpy 1.26.4

   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
    Uninstalling numpy-1.26.4:
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
      Successfully uninstalled numpy-1.26.4
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   -

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.16.1 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.2.6 which is incompatible.


In [3]:
pip install mediapipe

  Using cached mediapipe-0.10.21-cp311-cp311-win_amd64.whl.metadata (10 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sentencepiece-0.2.1-cp311-cp311-win_amd64.whl.metadata (10 kB)
  Using cached ml_dtypes-0.5.3-cp311-cp311-win_amd64.whl.metadata (9.2 kB)
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
  Using cached jax-0.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.7.1-cp311-cp311-win_amd64.whl.metadata (1.3 kB)
  Using cached contourpy-1.3.3-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp311-cp311-win_amd64.whl.metadata (6.4 kB)
  Using cached pillow-12.0.0-cp311-cp311-win_amd64.whl.metadata (9.0 kB)
INFO: pip is looking at multiple versions of opencv-c

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tensorflow-intel 2.16.1 requires ml-dtypes~=0.3.1, but you have ml-dtypes 0.5.3 which is incompatible.


In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
import pandas as pd

# SETUP MEDIAPIPE POSE
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

# DATASET PATH

DATASET_DIR = r"dataset/TEST"


# ---------------------------
# OUTPUT CSV
# ---------------------------
output_rows = []

# ---------------------------
# FUNCTION: Extract keypoints
# ---------------------------
def extract_landmarks(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if not results.pose_landmarks:
        return None

    landmarks = results.pose_landmarks.landmark

    # Store (x, y) of all 33 joints → 66 values
    keypoints = []
    for lm in landmarks:
        keypoints.append(lm.x)
        keypoints.append(lm.y)

    return keypoints

# ---------------------------
# SCAN ALL IMAGES
# ---------------------------
classes = os.listdir(DATASET_DIR)

for label in classes:
    folder = os.path.join(DATASET_DIR, label)
    if not os.path.isdir(folder):
        continue

    print(f"📌 Processing class: {label}")

    for filename in os.listdir(folder):
        if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)

        if img is None:
            print("Skipping unreadable image:", img_path)
            continue

        keypoints = extract_landmarks(img)

        if keypoints is None:
            print("⚠ No person detected:", img_path)
            continue

        # Add label at the end
        keypoints.append(label)

        output_rows.append(keypoints)

# ---------------------------
# SAVE TO CSV
# ---------------------------
columns = [f"x{i}" for i in range(33)] + [f"y{i}" for i in range(33)] + ["label"]
df = pd.DataFrame(output_rows, columns=columns)

df.to_csv("pose_keypoints_dataset.csv", index=False)
print("✅ Keypoints saved → pose_keypoints_dataset.csv")
print("Total samples:", len(df))


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\project\\Yoga_pose_estimation_cv\\dataset\\TEST'